In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint

pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [2]:
# One hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [3]:
# Basic RNN
with tf.variable_scope('one_cell') as scope:
    # one cell RNN input_dim(4) => output_dim(2)
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units = hidden_size)
    print(cell.output_size, cell.state_size)
    
    x_data = np.array([[h]], dtype=np.float32)   # x_data = [[[1,0,0,0]]]
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(2, 2)
array([[[ 1.,  0.,  0.,  0.]]], dtype=float32)
array([[[-0.06476866,  0.62179518]]], dtype=float32)


In [4]:
with tf.variable_scope('two_sequences') as scope:
    # One cell RNN input_dim(4) -> output_dim(2) sequence : 5
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(1, 5, 4)
array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]], dtype=float32)


array([[[-0.69384813,  0.51584738],
        [ 0.89965731,  0.2087978 ],
        [-0.77162617, -0.52418882],
        [ 0.00532959,  0.02049727],
        [-0.14608444,  0.68621504]]], dtype=float32)


In [6]:
# LSTM
with tf.variable_scope('3_batches') as scope:
    # One cell RNN input_dim(4) --> output_dim(2)
    # 3 batches 'hello', 'eolll', lleel'
    x_data = np.array([[h,e,l,l,o], [e,o,l,l,l], [l,l,e,e,l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)


array([[[-0.08646134, -0.14789675],
        [-0.11625829, -0.08102793],
        [-0.07494396, -0.08335794],
        [-0.00976982, -0.07909626],
        [-0.09661489, -0.17469703]],

       [[-0.0609024 , -0.00932112],
        [-0.16642259, -0.11906125],
        [-0.05380958, -0.09322259],
        [ 0.0036958 , -0.08442853],
        [ 0.03586017, -0.07910772]],

       [[ 0.02662396, -0.02678973],
        [ 0.04388273, -0.04363941],
        [-0.03749275, -0.03422633],
        [-0.0840456 , -0.0314363 ],
        [-0.04638144, -0.0490476 ]]], dtype=float32)


In [7]:
# LSTM 
with tf.variable_scope('3_batches_dynamic_length') as scope:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    # dynamic rnn => if sequence_length is shorter than sequence(5), it will be filled by 0. 
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)  
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)


array([[[ 0.13909647,  0.0110114 ],
        [-0.01803292,  0.00332305],
        [-0.09104887,  0.02952678],
        [-0.14116226,  0.05140863],
        [ 0.08342195, -0.02459829]],

       [[-0.11848822, -0.00168677],
        [ 0.08892012, -0.07528776],
        [-0.00996376, -0.02663   ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[-0.07610425,  0.02738276],
        [-0.13059093,  0.04953663],
        [-0.19947572,  0.05074293],
        [-0.25693089,  0.04047839],
        [ 0.        ,  0.        ]]], dtype=float32)


In [8]:
with tf.variable_scope('initial_state') as scope:
    batch_size = 3
    x_data = np.array([[h, e, l, l, o],
                      [e, o, l, l, l],
                      [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch: 3
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, initial_state=initial_state, dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)


array([[[-0.0413437 ,  0.08118443],
        [ 0.00756459,  0.10697021],
        [ 0.12485246,  0.14721324],
        [ 0.21304998,  0.1933749 ],
        [ 0.12865824,  0.3393003 ]],

       [[ 0.04879317,  0.07667817],
        [ 0.02099967,  0.14828235],
        [ 0.14183298,  0.14614041],
        [ 0.22531234,  0.19547103],
        [ 0.2821697 ,  0.22906584]],

       [[ 0.11347404,  0.08506513],
        [ 0.20226583,  0.15165241],
        [ 0.23946199,  0.20803738],
        [ 0.25563726,  0.22878005],
        [ 0.28760272,  0.22903199]]], dtype=float32)


In [11]:
# Create input data 
batch_size = 3          # one sentence
sequence_length = 5     # /ihello/ == 6
input_dim = 3           # one-hot size

x_data = np.arange(45, dtype=np.float32).reshape(batch_size, sequence_length, input_dim)
pp.pprint(x_data)   # batch, sequence_length, input_dim

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.],
        [ 12.,  13.,  14.]],

       [[ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.],
        [ 24.,  25.,  26.],
        [ 27.,  28.,  29.]],

       [[ 30.,  31.,  32.],
        [ 33.,  34.,  35.],
        [ 36.,  37.,  38.],
        [ 39.,  40.,  41.],
        [ 42.,  43.,  44.]]], dtype=float32)


In [12]:
with tf.variable_scope('generated_data') as scope:
    # One cell RNN input_dim (3) -> output_dim (5). sequence: 5, batch: 3
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ -7.92919695e-02,   1.85162619e-01,  -7.31386915e-02,
           4.35142824e-03,   9.28401425e-02],
        [ -3.89437675e-01,   4.50036615e-01,  -8.91992450e-02,
           4.52639954e-03,   1.75075039e-01],
        [ -3.26699108e-01,   6.31045818e-01,  -5.54964468e-02,
           3.76502867e-03,   1.98012620e-01],
        [ -1.60800010e-01,   6.94931090e-01,  -2.36288738e-02,
           3.16178612e-03,   1.82859302e-01],
        [ -7.16146007e-02,   7.12133586e-01,  -7.65812164e-03,
           2.73896358e-03,   1.49281561e-01]],

       [[ -4.19602171e-02,   5.25127172e-01,  -4.48014122e-04,
           1.47606416e-09,   5.16301878e-02],
        [ -1.81424376e-02,   7.02774465e-01,  -2.03356612e-04,
           1.12029985e-09,   6.25356138e-02],
        [ -8.36066809e-03,   7.43161142e-01,  -6.32645606e-05,
           9.19378462e-10,   5.21088392e-02],
        [ -4.05709445e-03,   7.59028912e-01,  -1.65625297e-05,
           7.89288135e-10,   3.78354453e-02],
        [ -1.9879

In [30]:
with tf.variable_scope('MultiRNNCell1') as scope: 
    # Make rnn
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True) # hidden_size = output size (5)   
    cell = rnn.MultiRNNCell([cell] * 3, state_is_tuple=True) # 3 layers

    # rnn in/out
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

ValueError: Variable MultiRNNCell1/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-29-37705e37548a>", line 7, in <module>
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
  File "/Users/hlee/Library/Python/2.7/lib/python/site-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/hlee/Library/Python/2.7/lib/python/site-packages/IPython/core/interactiveshell.py", line 2822, in run_ast_nodes
    if self.run_code(code, result):


In [31]:

with tf.variable_scope('dynamic_rnn') as scope:
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32,
                                         sequence_length=[1, 3, 2])
    # lentgh 1 for batch 1, lentgh 2 for batch 2
    
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

('dynamic rnn: ', <tf.Tensor 'dynamic_rnn/rnn/transpose:0' shape=(3, 5, 5) dtype=float32>)


ValueError: Fetch argument <tf.Operation 'dynamic_rnn/init' type=NoOp> cannot be interpreted as a Tensor. (Operation name: "dynamic_rnn/init"
op: "NoOp"
input: "^MultiRNNCell/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias/Assign"
input: "^MultiRNNCell/rnn/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell/rnn/basic_lstm_cell/bias/Assign"
input: "^MultiRNNCell1/rnn/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell1/rnn/basic_lstm_cell/bias/Assign"
input: "^MultiRNNCell1/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell1/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias/Assign"
input: "^dynamic_rnn/rnn/basic_lstm_cell/kernel/Assign"
input: "^dynamic_rnn/rnn/basic_lstm_cell/bias/Assign"
 is not an element of this graph.)

In [32]:
with tf.variable_scope('bi-directional') as scope:
    # bi-directional rnn
    cell_fw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell_bw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)

    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                      sequence_length=[2, 3, 1],
                                                      dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(sess.run(outputs))
    pp.pprint(sess.run(states))

ValueError: Fetch argument <tf.Operation 'bi-directional/init' type=NoOp> cannot be interpreted as a Tensor. (Operation name: "bi-directional/init"
op: "NoOp"
input: "^MultiRNNCell/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias/Assign"
input: "^MultiRNNCell/rnn/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell/rnn/basic_lstm_cell/bias/Assign"
input: "^MultiRNNCell1/rnn/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell1/rnn/basic_lstm_cell/bias/Assign"
input: "^MultiRNNCell1/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell1/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias/Assign"
input: "^dynamic_rnn/rnn/basic_lstm_cell/kernel/Assign"
input: "^dynamic_rnn/rnn/basic_lstm_cell/bias/Assign"
input: "^bi-directional/bidirectional_rnn/fw/basic_lstm_cell/kernel/Assign"
input: "^bi-directional/bidirectional_rnn/fw/basic_lstm_cell/bias/Assign"
input: "^bi-directional/bidirectional_rnn

In [33]:
# flattern based softmax
hidden_size=3
sequence_length=5
batch_size=3
num_classes=5

pp.pprint(x_data) # hidden_size=3, sequence_length=4, batch_size=2
x_data = x_data.reshape(-1, hidden_size)
pp.pprint(x_data)

softmax_w = np.arange(15, dtype=np.float32).reshape(hidden_size, num_classes)
outputs = np.matmul(x_data, softmax_w)
outputs = outputs.reshape(-1, sequence_length, num_classes) # batch, seq, class
pp.pprint(outputs)

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.],
        [ 12.,  13.,  14.]],

       [[ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.],
        [ 24.,  25.,  26.],
        [ 27.,  28.,  29.]],

       [[ 30.,  31.,  32.],
        [ 33.,  34.,  35.],
        [ 36.,  37.,  38.],
        [ 39.,  40.,  41.],
        [ 42.,  43.,  44.]]], dtype=float32)
array([[  0.,   1.,   2.],
       [  3.,   4.,   5.],
       [  6.,   7.,   8.],
       [  9.,  10.,  11.],
       [ 12.,  13.,  14.],
       [ 15.,  16.,  17.],
       [ 18.,  19.,  20.],
       [ 21.,  22.,  23.],
       [ 24.,  25.,  26.],
       [ 27.,  28.,  29.],
       [ 30.,  31.,  32.],
       [ 33.,  34.,  35.],
       [ 36.,  37.,  38.],
       [ 39.,  40.,  41.],
       [ 42.,  43.,  44.]], dtype=float32)
array([[[   25.,    28.,    31.,    34.,    37.],
        [   70.,    82.,    94.,   106.,   118.],
        [  115.,   136.,   157.,   178

In [34]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim ]
prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32)

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weights)
sess.run(tf.global_variables_initializer())
print("Loss: ", sequence_loss.eval())

ValueError: Fetch argument <tf.Operation 'init' type=NoOp> cannot be interpreted as a Tensor. (Operation name: "init"
op: "NoOp"
input: "^MultiRNNCell/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias/Assign"
input: "^MultiRNNCell/rnn/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell/rnn/basic_lstm_cell/bias/Assign"
input: "^MultiRNNCell1/rnn/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell1/rnn/basic_lstm_cell/bias/Assign"
input: "^MultiRNNCell1/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell1/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias/Assign"
input: "^dynamic_rnn/rnn/basic_lstm_cell/kernel/Assign"
input: "^dynamic_rnn/rnn/basic_lstm_cell/bias/Assign"
input: "^bi-directional/bidirectional_rnn/fw/basic_lstm_cell/kernel/Assign"
input: "^bi-directional/bidirectional_rnn/fw/basic_lstm_cell/bias/Assign"
input: "^bi-directional/bidirectional_rnn/bw/basic_lstm_cell/kernel/Ass

In [35]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim ]
prediction1 = tf.constant([[[0.3, 0.7], [0.3, 0.7], [0.3, 0.7]]], dtype=tf.float32)
prediction2 = tf.constant([[[0.1, 0.9], [0.1, 0.9], [0.1, 0.9]]], dtype=tf.float32)

prediction3 = tf.constant([[[1, 0], [1, 0], [1, 0]]], dtype=tf.float32)
prediction4 = tf.constant([[[0, 1], [1, 0], [0, 1]]], dtype=tf.float32)

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)
sequence_loss4 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
      "Loss2: ", sequence_loss2.eval(),
      "Loss3: ", sequence_loss3.eval(),
      "Loss4: ", sequence_loss4.eval())

ValueError: Fetch argument <tf.Operation 'init_1' type=NoOp> cannot be interpreted as a Tensor. (Operation name: "init_1"
op: "NoOp"
input: "^MultiRNNCell/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias/Assign"
input: "^MultiRNNCell/rnn/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell/rnn/basic_lstm_cell/bias/Assign"
input: "^MultiRNNCell1/rnn/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell1/rnn/basic_lstm_cell/bias/Assign"
input: "^MultiRNNCell1/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel/Assign"
input: "^MultiRNNCell1/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias/Assign"
input: "^dynamic_rnn/rnn/basic_lstm_cell/kernel/Assign"
input: "^dynamic_rnn/rnn/basic_lstm_cell/bias/Assign"
input: "^bi-directional/bidirectional_rnn/fw/basic_lstm_cell/kernel/Assign"
input: "^bi-directional/bidirectional_rnn/fw/basic_lstm_cell/bias/Assign"
input: "^bi-directional/bidirectional_rnn/bw/basic_lstm_cell/kernel